In [10]:
%load_ext autoreload
%autoreload 2
import os
import fitz  # PyMuPDF
os.environ["OPENAI_API_KEY"] = ""

from raptor import RetrievalAugmentation, RetrievalAugmentationConfig

# Initialize with default configuration. For advanced configurations, check the documentation. [WIP]
custom_config = RetrievalAugmentationConfig(
    tb_max_tokens=300,  # Updated value
    tb_num_layers=5,    # Updated value
    tb_threshold=0.75,  # Updated value
    tb_top_k=8,         # Updated value
    tb_summarization_length=200,  # Updated value
    tr_threshold=0.75,  # Updated value
    tr_top_k=8,         # Updated value
    qa_model=None,      # Ensure the correct QA model is used
    embedding_model=None,  # Ensure the correct embedding model is used
    summarization_model=None  # Ensure the correct summarization model is used
)

# Initialize RetrievalAugmentation with the custom configuration
RA = RetrievalAugmentation(config=custom_config)

2025-03-19 10:45:41,535 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 300
            Num Layers: 5
            Threshold: 0.75
            Top K: 8
            Selection Mode: top_k
            Summarization Length: 200
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x36722a1a0>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x36722b460>}
            Cluster Embedding Model: OpenAI
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2025-03-19 10:45:41,536 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 300
            Num Layers: 5
            Threshold: 0.75
            Top K: 8
            Sel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!ls pdfs

clm104c01.pdf


In [6]:
# Load PDF and extract text
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n\n".join(page.get_text("text") for page in doc)
    return text

# Example usage
pdf_text = extract_text_from_pdf("./pdfs/clm104c13.pdf")

In [8]:
len(pdf_text)

175717

In [11]:
# Add extracted text to RAPTOR
RA.add_documents(pdf_text)

2025-03-19 10:45:48,944 - Creating Leaf Nodes
2025-03-19 10:45:49,640 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,673 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,695 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,702 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,707 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,711 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,727 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,733 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,755 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:45:49,779 - HTTP Request: POST https://a

In [7]:
question = "What is the relationship between payment jurisdictions and physician fee schedules?"
answer = RA.answer_question(question=question)
print("Answer: ", answer)

2025-03-19 09:03:44,645 - Using collapsed_tree
2025-03-19 09:03:45,718 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:03:52,938 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer:  The relationship between payment jurisdictions and physician fee schedules is crucial in determining how payment requests for services paid under the Medicare Physician Fee Schedule (MPFS) and anesthesia services are processed. Payment jurisdictions among A/B MACs (Part B) dictate which MAC is responsible for processing payment requests for specific services. This ensures that claims are handled by the appropriate MAC based on the services provided and the geographic location.

Physician fee schedules, on the other hand, establish the payment rates for various medical services provided by physicians, laboratories, and other healthcare providers. These fee schedules outline the reimbursement amounts that Medicare will pay for specific services based on the relative value units (RVUs) assigned to each service.

The relationship between payment jurisdictions and physician fee schedules is essential for ensuring accurate and timely payment processing for services rendered under th

In [8]:
q2 = "How does Medicare determine the correct jurisdiction for a reassigned service claim?"
answer2 = RA.answer_question(question=q2)
print("Answer: ", answer2)

2025-03-19 09:04:37,399 - Using collapsed_tree
2025-03-19 09:04:38,327 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:04:40,804 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer:  Medicare determines the correct jurisdiction for a reassigned service claim based on specific guidelines outlined in the Program Integrity Manual. Providers and suppliers must bill the correct B/MAC (Medicare Administrative Contractor) for reassigned services, even if they were initially paid under the Medicare Physician Fee Schedule (MPFS). Claims must be submitted to the B/MAC that has jurisdiction over the geographic area where the services were provided. Additionally, providers and suppliers must meet the current enrollment criteria specified in the Program Integrity Manual chapter 10 to bill for reassigned services. It is essential to submit claims to the appropriate A/B MACs for processing, especially if a healthcare provider operates in multiple localities or under multiple MACs for Part B services. Medicare follows specific rules and procedures to ensure that claims are processed correctly and efficiently, and misdirected claims are handled according to the specified g

In [9]:
SAVE_PATH = "./saved_trees/default"
RA.save(SAVE_PATH)

2025-03-19 09:33:43,585 - Tree successfully saved to ./saved_trees/default


In [21]:
import pandas as pd
import openai
import json

In [22]:
questions = [
    "What are the electronic submission requirements for Medicare claims?",
    "What HIPAA standards apply to Medicare claims processing?",
    "Where can I purchase HIPAA standard implementation guides?",
    "What are the paper format requirements for institutional claims?",
    "What information should be included in remittance advices?",
    "How does Medicare determine the correct jurisdiction for a payment request?",
    "What are the payment jurisdiction rules for physician fee schedules?",
    "What exceptions exist for jurisdictional claims processing for DME suppliers?",
    "How are ambulance services submitted to carriers?",
    "How are independent laboratory claims processed?",
    "What transmittals have been issued for Chapter 1?",
    "What are the latest updates in Medicare’s general billing requirements?",
    "What are the most common claim rejection reasons under this manual?",
    "What should be included in a Medicare claim to avoid rejections?",
    "What is the process for submitting a reconsideration request for a denied Medicare claim?"
]

# 🔹 Store Q/A Pairs
qa_results = []

# 🔹 Query RAPTOR and Store Results
for question in questions:
    answer = RA.answer_question(question=question)
    qa_results.append({"question": question, "retrieved_answer": answer})

# 🔹 Initialize OpenAI Client
client = openai.OpenAI()

# 🔹 Evaluate Each Q/A Pair Using GPT-4o-mini
for qa in qa_results:
    prompt = f"""
    Evaluate the following Question & Answer pair based on:

    - **Accuracy (0-10):** Does the answer correctly address the question based on Medicare's official guidelines?
    - **Completeness (0-10):** Does the answer include all relevant details?
    - **Faithfulness (0-10):** Does the answer strictly use information from the document without hallucination?
    - **Relevance (0-10):** Does the answer stay focused on the question, or does it introduce unrelated/extraneous information?
    - **Feedback:** Provide a brief explanation of the scores and suggest improvements.

    **respond ONLY with a JSON object** in the format:
    {{"feedback": str, "metrics": {{"accuracy": int, "completeness": int, "faithfulness": int, "relevance": int}}}}

    **Do NOT include markdown, code blocks, or explanations outside the JSON response.**  
    We will **directly parse your output**, so it must be a **valid JSON object**.


    ---  
    **Question:** {qa["question"]}
    **Retrieved Answer:** {qa["retrieved_answer"]}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 🔹 Use the latest model for evaluation
        messages=[{"role": "system", "content": "You are an expert evaluator of Medicare compliance documents. Return a JSON object only."},
                  {"role": "user", "content": prompt}],
        temperature=0.0  # 🔹 Ensures deterministic, consistent evaluations
    )

    evaluation_text = response.choices[0].message.content.strip()

    # 🔹 Attempt to parse JSON safely
    try:
        evaluation_data = json.loads(evaluation_text)

        # Ensure expected structure exists
        if "metrics" in evaluation_data and all(
            key in evaluation_data["metrics"] for key in ["accuracy", "completeness", "faithfulness", "relevance"]
        ):
            qa.update({
                "accuracy": evaluation_data["metrics"]["accuracy"],
                "completeness": evaluation_data["metrics"]["completeness"],
                "faithfulness": evaluation_data["metrics"]["faithfulness"],
                "relevance": evaluation_data["metrics"]["relevance"],
                "feedback": evaluation_data["feedback"]
            })
        else:
            raise ValueError("Missing expected JSON keys in response.")

    except Exception as e:
        qa.update({
            "accuracy": None,
            "completeness": None,
            "faithfulness": None,
            "relevance": None,
            "feedback": f"Error parsing evaluation: {str(e)}"
        })

# 🔹 Convert Results to DataFrame
df_results = pd.DataFrame(qa_results)

2025-03-19 09:50:04,888 - Using collapsed_tree
2025-03-19 09:50:05,709 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:50:07,595 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 09:50:07,601 - Using collapsed_tree
2025-03-19 09:50:07,925 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:50:09,598 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 09:50:09,607 - Using collapsed_tree
2025-03-19 09:50:09,888 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:50:10,962 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 09:50:10,968 - Using collapsed_tree
2025-03-19 09:50:11,320 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 09:50:12,647 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/

In [25]:
df_results

,question,retrieved_answer,accuracy,completeness,faithfulness,relevance,feedback
0,What are the electronic submission requirement...,The electronic submission requirements for Med...,9,7,10,9,The answer accurately addresses the question r...
1,What HIPAA standards apply to Medicare claims ...,The HIPAA standards that apply to Medicare cla...,9,8,9,10,The answer accurately addresses the question b...
2,Where can I purchase HIPAA standard implementa...,You can purchase HIPAA standard implementation...,9,7,9,8,The answer provides accurate sources for purch...
3,What are the paper format requirements for ins...,The paper format requirements for institutiona...,9,7,10,9,The answer accurately addresses the question r...
4,What information should be included in remitta...,The information that should be included in rem...,9,8,9,7,The answer provides a comprehensive overview o...
5,How does Medicare determine the correct jurisd...,Medicare determines the correct jurisdiction f...,9,8,9,8,The answer accurately addresses the question b...
6,What are the payment jurisdiction rules for ph...,The payment jurisdiction rules for physician f...,8,7,9,8,The answer provides a detailed overview of the...
7,What exceptions exist for jurisdictional claim...,The exceptions for jurisdictional claims proce...,8,6,9,7,The answer provides a good overview of the exc...
8,How are ambulance services submitted to carriers?,Ambulance services are submitted to carriers b...,9,8,10,9,The answer provides a detailed explanation of ...
9,How are independent laboratory claims processed?,Independent laboratory claims are processed ba...,9,8,9,10,The answer accurately addresses the question r...


In [13]:
from evaluator import RAPTOREvaluator

questions_answers = [
    {
        "question": "What is the main focus of Chapter 13 in the Medicare Claims Processing Manual?",
        "example_answer": "Chapter 13 focuses on Radiology Services and Other Diagnostic Procedures."
    },
    {
        "question": "Which section of Chapter 13 covers ICD Coding for Diagnostic Tests?",
        "example_answer": "The section titled '10 - ICD Coding for Diagnostic Tests' provides coding guidelines for diagnostic tests in radiology."
    },
    {
        "question": "What is the difference between the Professional Component (PC) and the Technical Component (TC) in radiology billing?",
        "example_answer": "The Professional Component (PC) refers to the interpretation and reporting by the radiologist, while the Technical Component (TC) covers the use of equipment and the execution of the imaging procedure."
    },
    {
        "question": "What information is provided under 'Payment Conditions for Radiology Services' in Chapter 13?",
        "example_answer": "It outlines the billing rules, payment conditions, and reimbursement guidelines for radiology services, including details for both professional and technical components."
    },
    {
        "question": "How are radiology services provided in leased departments treated under Medicare billing rules?",
        "example_answer": "Services furnished in leased hospital departments are subject to special billing instructions that differentiate them from in-house hospital services, affecting both reimbursement and claim processing."
    },
    {
        "question": "What is the purpose of the 'Special Rule to Incentivize Transition from Traditional X-Ray Imaging to Digital Radiography'?",
        "example_answer": "This rule is designed to encourage the adoption of digital radiography by providing financial incentives, thereby promoting more modern imaging technologies over traditional film-based methods."
    },
    {
        "question": "How does Medicare handle billing for radiology services that are not furnished in hospitals?",
        "example_answer": "Radiology services provided outside of hospitals are billed under the Medicare Physician Fee Schedule with specific guidelines that account for the different service settings."
    },
    {
        "question": "What HCPCS codes are designated for Low Osmolar Contrast Media (LOCM) in Chapter 13?",
        "example_answer": "The HCPCS codes for LOCM are Q9945 through Q9951."
    },
    {
        "question": "What payment criteria are outlined for Low Osmolar Contrast Media (LOCM) in Chapter 13?",
        "example_answer": "Payment for LOCM is based on clinical criteria, proper use of contrast media, and may include reductions if non-compliant imaging equipment is used."
    },
    {
        "question": "What are the key billing instructions for Magnetic Resonance Imaging (MRI) Procedures in this chapter?",
        "example_answer": "The billing instructions for MRI include specific coding requirements, the use of appropriate modifiers, and guidelines for differentiating between technical and professional components."
    },
    {
        "question": "How does Magnetic Resonance Angiography (MRA) differ from standard MRI as outlined in Chapter 13?",
        "example_answer": "MRA is specifically focused on imaging blood vessels and requires additional coding and coverage criteria compared to standard MRI, which covers a broader range of imaging procedures."
    },
    {
        "question": "What is the significance of modifier codes such as KX and FX in radiology billing?",
        "example_answer": "Modifiers like KX and FX indicate special billing circumstances or adjustments, such as confirming that a service was performed under specific conditions, which in turn can affect the payment amount."
    },
    {
        "question": "How does Chapter 13 address the anti-markup payment limitation for diagnostic tests?",
        "example_answer": "It sets forth rules that limit payment to the lowest of several amounts—such as the provider’s net charge or the fee schedule amount—ensuring that excessive markups are not applied."
    },
    {
        "question": "What are the potential consequences of not including required line item details in a radiology claim?",
        "example_answer": "Failure to include necessary details like revenue codes, service dates, or proper modifiers can lead to claim denials, payment reductions, or delays in processing."
    },
    {
        "question": "How are Payment Conditions for PET scans addressed in Chapter 13?",
        "example_answer": "The chapter provides detailed billing and coverage guidelines for PET scans, specifying the appropriate CPT/HCPCS codes, modifiers, and conditions under which PET scans are reimbursed."
    },
    {
        "question": "What special billing instructions are provided for PET scans related to conditions such as myocardial viability or breast cancer?",
        "example_answer": "Special instructions include using designated CPT codes and modifiers, along with specific coverage criteria that ensure the PET scan is justified for conditions like myocardial viability or breast cancer."
    },
    {
        "question": "How does Chapter 13 differentiate between diagnostic and therapeutic nuclear medicine services?",
        "example_answer": "Diagnostic nuclear medicine services are billed separately based on the imaging procedure and radiopharmaceuticals used, whereas therapeutic nuclear medicine services may be bundled with other treatment costs and have their own coding requirements."
    },
    {
        "question": "Explain the use of CPT Modifier '-51' in the context of nuclear medicine billing.",
        "example_answer": "CPT Modifier '-51' is applied when multiple procedures are performed on the same day, indicating that a reduction should be applied to the overall payment as per the multiple procedure policy."
    },
    {
        "question": "What role do Medicare Summary Notices (MSN) play in the radiology claims process as described in Chapter 13?",
        "example_answer": "MSNs provide detailed feedback on submitted claims, including explanations for denials or adjustments, and help both providers and contractors reconcile billing discrepancies."
    },
    {
        "question": "Analyze how Chapter 13 integrates billing instructions across various modalities (CT, MRI, PET, nuclear medicine) to create a cohesive framework for radiology services.",
        "example_answer": "Chapter 13 unifies diverse billing rules and coding standards by offering a comprehensive set of guidelines that span multiple imaging modalities. This integration ensures consistency in claim processing, proper reimbursement, and compliance with Medicare regulations across different types of radiology services."
    }
]

evaluator = RAPTOREvaluator(questions_answers, RA)
df_evaluations = evaluator.evaluate()
print(df_evaluations)
# Query RAPTOR and store results by retrieving answers for each question.


2025-03-19 10:57:43,719 - Using collapsed_tree
2025-03-19 10:57:44,642 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:57:46,683 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 10:57:48,286 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 10:57:48,296 - Using collapsed_tree
2025-03-19 10:57:49,034 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:57:49,765 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 10:57:50,782 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 10:57:50,785 - Using collapsed_tree
2025-03-19 10:57:51,345 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 10:57:53,563 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 10:57:55,457 - HTT

                                             question  \
0   What is the main focus of Chapter 13 in the Me...   
1   Which section of Chapter 13 covers ICD Coding ...   
2   What is the difference between the Professiona...   
3   What information is provided under 'Payment Co...   
4   How are radiology services provided in leased ...   
5   What is the purpose of the 'Special Rule to In...   
6   How does Medicare handle billing for radiology...   
7   What HCPCS codes are designated for Low Osmola...   
8   What payment criteria are outlined for Low Osm...   
9   What are the key billing instructions for Magn...   
10  How does Magnetic Resonance Angiography (MRA) ...   
11  What is the significance of modifier codes suc...   
12  How does Chapter 13 address the anti-markup pa...   
13  What are the potential consequences of not inc...   
14  How are Payment Conditions for PET scans addre...   
15  What special billing instructions are provided...   
16  How does Chapter 13 differe

In [29]:
df_results.to_csv("./results/default_results1.csv")

In [14]:
df_evaluations

,question,example_answer,retrieved_answer,accuracy,completeness,faithfulness,relevance,feedback
0,What is the main focus of Chapter 13 in the Me...,Chapter 13 focuses on Radiology Services and O...,The main focus of Chapter 13 in the Medicare C...,10,10,10,10,The retrieved answer accurately addresses the ...
1,Which section of Chapter 13 covers ICD Coding ...,The section titled '10 - ICD Coding for Diagno...,The section of Chapter 13 that covers ICD Codi...,10,10,10,10,The retrieved answer accurately identifies the...
2,What is the difference between the Professiona...,The Professional Component (PC) refers to the ...,The Professional Component (PC) and the Techni...,9,8,10,10,The retrieved answer accurately describes the ...
3,What information is provided under 'Payment Co...,"It outlines the billing rules, payment conditi...",The information provided under 'Payment Condit...,10,9,10,10,The retrieved answer provides a detailed and a...
4,How are radiology services provided in leased ...,Services furnished in leased hospital departme...,Radiology services provided in leased hospital...,9,8,9,8,The retrieved answer accurately addresses the ...
5,What is the purpose of the 'Special Rule to In...,This rule is designed to encourage the adoptio...,The purpose of the 'Special Rule to Incentiviz...,10,10,10,10,The retrieved answer accurately addresses the ...
6,How does Medicare handle billing for radiology...,Radiology services provided outside of hospita...,Medicare handles billing for radiology service...,9,9,10,9,The retrieved answer provides a detailed expla...
7,What HCPCS codes are designated for Low Osmola...,The HCPCS codes for LOCM are Q9945 through Q9951.,The HCPCS codes designated for Low Osmolar Con...,10,10,10,10,The retrieved answer accurately reflects the i...
8,What payment criteria are outlined for Low Osm...,Payment for LOCM is based on clinical criteria...,The payment criteria outlined for Low Osmolar ...,9,9,10,8,The retrieved answer provides a detailed overv...
9,What are the key billing instructions for Magn...,The billing instructions for MRI include speci...,The key billing instructions for Magnetic Reso...,8,7,9,9,The retrieved answer provides detailed billing...


In [18]:
import openai
import json
import pandas as pd

class ClaimAssessor:
    def __init__(self, ra_instance, model="gpt-4o-mini", temperature=0.0):
        """
        Initializes the ClaimAssessor.
        Parameters:
          - ra_instance: An instance of RAPTOR (RetrievalAugmentation) used to query the rules corpus.
          - model: The OpenAI model to use for all LLM calls.
          - temperature: The temperature for API calls.
        """
        self.ra = ra_instance
        self.model = model
        self.temperature = temperature
        self.client = openai.OpenAI()

    def expand_claim_to_queries(self, claim):
        """
        Uses an LLM call to convert a raw claim into one or more queries.
        Returns a list of query strings.
        """
        prompt = f"""
You are an expert in Medicare radiology rules (Chapter 13). Given the following claim details, generate a JSON object with a key "queries" whose value is a list of one or more natural language queries that can be used to search the rules corpus. Your queries should reference the procedure description, relevant CPT and ICD codes, and any financial or coding details that might impact claim acceptance.

Claim details:
ID: {claim.get('id')}
Procedure Description: {claim.get('procedure_description')}
Place of Service: {claim.get('place_of_service')}
Service Type: {claim.get('service_type')}
CPT Codes: {', '.join(claim.get('cpt_codes', []))}
ICD Codes: {', '.join(claim.get('icd_codes', []))}
Charge Amount: {claim.get('charge_amount')}
Status: {claim.get('status')}
Pre-Auth Status: {claim.get('pre_auth_status')}

Respond ONLY with a JSON object in the format:
{{"queries": ["query1", "query2", ...]}}
Do NOT include any additional text.
"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in generating precise search queries for Medicare Chapter 13 rules."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature
        )
        output = response.choices[0].message.content.strip()
        try:
            data = json.loads(output)
            queries = data.get("queries", [])
            return queries
        except Exception as e:
            print(f"Error parsing query expansion output: {e}")
            return []

    def retrieve_rules_by_query(self, queries):
        """
        For each query, uses the RAPTOR agent to retrieve the relevant rule text.
        Returns a dictionary mapping each query to its retrieved text.
        """
        query_results = {}
        for query in queries:
            rule_text = self.ra.answer_question(question=query)
            query_results[query] = rule_text
        return query_results

    def evaluate_query_pair(self, query, retrieved_text):
        """
        Evaluates a single query-answer pair using an LLM call.
        Returns a dictionary with metrics: accuracy, completeness, faithfulness, relevance and feedback.
        (In this context, metrics capture how well the retrieved rule text answers the query.)
        """
        prompt = f"""
Evaluate the following query and its retrieved answer based on Medicare Chapter 13 rules. 
Provide scores (0-10) for:
- Accuracy: How correct is the retrieved text in addressing the query?
- Completeness: How complete is the information relative to the query?
- Faithfulness: How strictly does the text reflect the actual rules (without hallucination)?
- Relevance: How focused is the answer on the query?

Respond ONLY with a JSON object in the format:
{{"feedback": str, "metrics": {{"accuracy": int, "completeness": int, "faithfulness": int, "relevance": int}}}}
Do NOT include any additional text.
---
Query: {query}
Retrieved Answer: {retrieved_text}
"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert evaluator of Medicare Chapter 13 rules. Return a JSON object only."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature
        )
        output = response.choices[0].message.content.strip()
        keys = ["accuracy", "completeness", "faithfulness", "relevance"]
        try:
            data = json.loads(output)
            if "metrics" in data and all(key in data["metrics"] for key in keys):
                print(f"Q/A quality average: {sum([float(data['metrics'][k]) for k in keys])/4.0}")
                return data
            else:
                print("exception was raised in the thing")
                raise ValueError("Missing expected keys.")
        except Exception as e:
            return {"feedback": f"Error parsing query evaluation: {str(e)}", "metrics": {"accuracy": None, "completeness": None, "faithfulness": None, "relevance": None}}

    def assess_claim(self, claim):
        """
        Processes a single claim:
         1. Expands the claim into queries.
         2. Retrieves rule text for each query.
         3. Evaluates each query-answer pair.
         4. Combines retrieved rules.
         5. Makes a final LLM call to assess the claim.
        Returns a dictionary containing:
         - The original claim.
         - Generated queries.
         - Query metrics (a list of dicts for each query).
         - Combined retrieved rules.
         - Final assessment (summary, acceptance probability, explanation).
        """
        # Step 1: Expand claim to queries.
        queries = self.expand_claim_to_queries(claim)
        if not queries:
            return {"claim": claim, "error": "No queries generated."}

        # Step 2: Retrieve rules for each query.
        query_to_rule = self.retrieve_rules_by_query(queries)
        
        # Step 3: Evaluate each query pair.
        query_metrics = []
        for query, rule_text in query_to_rule.items():
            metrics = self.evaluate_query_pair(query, rule_text)
            query_metrics.append({
                "query": query,
                "retrieved_answer": rule_text,
                "evaluation": metrics
            })

        # Step 4: Combine all retrieved rule texts.
        combined_rules = "\n\n".join(query_to_rule.values())

        # Step 5: Final LLM call to assess the claim.
        final_prompt = f"""
You are an expert in Medicare radiology rules (Chapter 13). Using the following claim details and the combined retrieved rules information, determine the likelihood that the claim will be accepted. Provide a summary of the relevant rules, a predicted acceptance probability (0 to 100 percent), and a clear, user-friendly explanation with any suggestions for next steps or indicate if there is insufficient information.

Claim Details:
ID: {claim.get('id')}
Procedure Description: {claim.get('procedure_description')}
Place of Service: {claim.get('place_of_service')}
Service Type: {claim.get('service_type')}
CPT Codes: {', '.join(claim.get('cpt_codes', []))}
ICD Codes: {', '.join(claim.get('icd_codes', []))}
Charge Amount: {claim.get('charge_amount')}
Status: {claim.get('status')}
Pre-Auth Status: {claim.get('pre_auth_status')}

Retrieved Rules Information:
{combined_rules}

Respond ONLY with a JSON object in the format:
{{"summary": str, "acceptance_probability": int, "explanation": str}}
Do NOT include any extra text.
"""
        final_response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert evaluator of Medicare radiology claims. Return a JSON object only."},
                {"role": "user", "content": final_prompt}
            ],
            temperature=self.temperature
        )
        final_output = final_response.choices[0].message.content.strip()
        try:
            final_data = json.loads(final_output)
        except Exception as e:
            final_data = {"error": f"Error parsing final assessment output: {str(e)}"}
        
        # Return all components.
        return {
            "claim": claim,
            "generated_queries": queries,
            "query_metrics": query_metrics,
            "combined_retrieved_rules": combined_rules,
            "final_assessment": final_data
        }

# ---------------------------------------------------------
# Sample Claims (including those synthesized earlier and some extra tricky ones)
claims = [
    {
        "id": "CLM001",
        "account": 12345,
        "patient": "Patient A",
        "entry_date": "2023-11-01",
        "bill_date": "2023-11-02",
        "date_of_service": "2023-10-30",
        "procedure_description": "MRI of chest with contrast for evaluation of a lung nodule.",
        "place_of_service": "Hospital Outpatient",
        "service_type": "MRI",
        "units": 1,
        "provider": "Radiology Clinic A",
        "facility": "Hospital A",
        "cpt_codes": ["70551"],
        "icd_codes": ["R91.8"],
        "charge_amount": 1500.00,
        "allowed_amount": 1200.00,
        "status": "submitted",
        "pre_auth_status": "approved",
        "is_flagged": False,
        "notes": []
    },
    {
        "id": "CLM002",
        "account": 12346,
        "patient": "Patient B",
        "entry_date": "2023-11-05",
        "bill_date": "2023-11-06",
        "date_of_service": "2023-11-03",
        "procedure_description": "CT scan of the abdomen without contrast for suspected appendicitis.",
        "place_of_service": "Hospital Inpatient",
        "service_type": "CT",
        "units": 1,
        "provider": "Radiology Clinic B",
        "facility": "Hospital B",
        "cpt_codes": ["74150"],
        "icd_codes": ["K35.80"],
        "charge_amount": 2000.00,
        "allowed_amount": 1800.00,
        "status": "submitted",
        "pre_auth_status": "pending",
        "is_flagged": True,
        "flagged_reason": "Missing required documentation for contrast use.",
        "notes": ["Needs review for compliance with contrast guidelines."]
    },
    {
        "id": "CLM003",
        "account": 12347,
        "patient": "Patient C",
        "entry_date": "2023-11-10",
        "bill_date": "2023-11-11",
        "date_of_service": "2023-11-09",
        "procedure_description": "PET scan for evaluation of metastatic breast cancer.",
        "place_of_service": "Hospital Outpatient",
        "service_type": "PET",
        "units": 1,
        "provider": "Radiology Clinic C",
        "facility": "Hospital C",
        "cpt_codes": ["78815"],
        "icd_codes": ["C50.911"],
        "charge_amount": 3000.00,
        "allowed_amount": 2500.00,
        "status": "submitted",
        "pre_auth_status": "approved",
        "is_flagged": False,
        "notes": ["Ensure compliance with PET billing guidelines."]
    },
    {
        "id": "CLM004",
        "account": 12348,
        "patient": "Patient D",
        "entry_date": "2023-11-15",
        "bill_date": "2023-11-16",
        "date_of_service": "2023-11-14",
        "procedure_description": "X-ray imaging using traditional film for evaluation of chronic knee pain.",
        "place_of_service": "Ambulatory Clinic",
        "service_type": "X-ray",
        "units": 1,
        "provider": "Radiology Clinic D",
        "facility": "Clinic D",
        "cpt_codes": ["73030"],
        "icd_codes": ["M17.11"],
        "charge_amount": 300.00,
        "allowed_amount": 250.00,
        "status": "submitted",
        "pre_auth_status": "not_required",
        "is_flagged": True,
        "flagged_reason": "Using outdated film-based imaging; digital recommended.",
        "notes": ["Consider upgrading to digital radiography as encouraged in Chapter 13."]
    },
    {
        "id": "CLM005",
        "account": 12349,
        "patient": "Patient E",
        "entry_date": "2023-11-20",
        "bill_date": "2023-11-21",
        "date_of_service": "2023-11-19",
        "procedure_description": "Digital radiography of the chest for evaluation of suspected pneumonia.",
        "place_of_service": "Hospital Outpatient",
        "service_type": "X-ray",
        "units": 1,
        "provider": "Radiology Clinic E",
        "facility": "Hospital E",
        "cpt_codes": ["71020"],
        "icd_codes": ["J18.9"],
        "charge_amount": 250.00,
        "allowed_amount": 220.00,
        "status": "submitted",
        "pre_auth_status": "not_required",
        "is_flagged": False,
        "notes": []
    },
    # Additional tricky claim examples:
    {
        "id": "CLM007",
        "account": 12350,
        "patient": "Patient F",
        "entry_date": "2023-12-01",
        "bill_date": "2023-12-02",
        "date_of_service": "2023-11-30",
        "procedure_description": "Computed radiography of the chest with ambiguous modifier usage and potential duplicate billing.",
        "place_of_service": "Hospital Outpatient",
        "service_type": "Computed Radiography",
        "units": 1,
        "provider": "Radiology Clinic F",
        "facility": "Hospital F",
        "cpt_codes": ["71010"],
        "icd_codes": ["J18.9"],
        "charge_amount": 400.00,
        "allowed_amount": 350.00,
        "status": "flagged",
        "pre_auth_status": "not_required",
        "is_flagged": True,
        "flagged_reason": "Ambiguous modifiers and duplicate billing risk.",
        "notes": ["Check modifier usage and ensure single billing."]
    },
    {
        "id": "CLM008",
        "account": 12351,
        "patient": "Patient G",
        "entry_date": "2023-12-05",
        "bill_date": "2023-12-06",
        "date_of_service": "2023-12-04",
        "procedure_description": "Digital radiography of the knee with ICD code mismatch for chronic pain.",
        "place_of_service": "Ambulatory Clinic",
        "service_type": "Digital Radiography",
        "units": 1,
        "provider": "Radiology Clinic G",
        "facility": "Clinic G",
        "cpt_codes": ["73030"],
        "icd_codes": ["M17.9"],  # Mismatch: M17.9 is less specific than required
        "charge_amount": 350.00,
        "allowed_amount": 300.00,
        "status": "submitted",
        "pre_auth_status": "not_required",
        "is_flagged": True,
        "flagged_reason": "ICD code may not fully justify the imaging for chronic knee pain.",
        "notes": ["Verify diagnosis coding against procedure."]
    }
]

# Process each claim using ClaimAssessor.
# (Ensure your RA_instance is properly configured; here we instantiate a dummy instance for illustration.)
RA_instance = RA  # Replace with your actual RAPTOR instance

assessor = ClaimAssessor(RA_instance)
results = []
for claim in claims:
    result = assessor.assess_claim(claim)
    results.append(result)

# Convert results into a DataFrame for analysis.
df_results = pd.DataFrame(results)

2025-03-19 11:27:32,011 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 11:27:32,019 - Using collapsed_tree
2025-03-19 11:27:32,549 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 11:27:36,315 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 11:27:36,319 - Using collapsed_tree
2025-03-19 11:27:36,991 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 11:27:40,917 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 11:27:40,924 - Using collapsed_tree
2025-03-19 11:27:41,416 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-19 11:27:45,145 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-19 11:27:47,370 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Q/A quality average: 4.0


2025-03-19 11:27:51,298 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Q/A quality average: 7.5


2025-03-19 11:27:53,515 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2025-03-19 11:31:11,315 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Q/A quality average: 7.0


2025-03-19 11:31:12,776 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Q/A quality average: 8.0


2025-03-19 11:31:15,564 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [19]:
df_results

,claim,generated_queries,query_metrics,combined_retrieved_rules,final_assessment
0,"{'id': 'CLM001', 'account': 12345, 'patient': ...",[What are the Medicare Chapter 13 rules for MR...,[{'query': 'What are the Medicare Chapter 13 r...,The Medicare Chapter 13 rules for MRI of the c...,{'summary': 'The claim for MRI of the chest wi...
1,"{'id': 'CLM002', 'account': 12346, 'patient': ...",[What are the Medicare coverage rules for CPT ...,[{'query': 'What are the Medicare coverage rul...,The Medicare coverage rules for CPT code 74150...,{'summary': 'The claim for a CT scan of the ab...
2,"{'id': 'CLM003', 'account': 12347, 'patient': ...",[What are the Medicare Chapter 13 rules for co...,[{'query': 'What are the Medicare Chapter 13 r...,The Medicare Chapter 13 rules for coverage of ...,{'summary': 'The claim for a PET scan (CPT 788...
3,"{'id': 'CLM004', 'account': 12348, 'patient': ...",[What are the Medicare rules for X-ray imaging...,[{'query': 'What are the Medicare rules for X-...,The Medicare rules for X-ray imaging using tra...,{'summary': 'Claims for X-rays using tradition...
4,"{'id': 'CLM005', 'account': 12349, 'patient': ...",[What are the Medicare Chapter 13 rules for CP...,[{'query': 'What are the Medicare Chapter 13 r...,The Medicare Chapter 13 rules for CPT code 710...,{'summary': 'The claim for CPT code 71020 (dig...
5,"{'id': 'CLM007', 'account': 12350, 'patient': ...",[What are the Medicare rules for billing CPT c...,[{'query': 'What are the Medicare rules for bi...,The Medicare rules for billing CPT code 71010 ...,{'summary': 'The claim for CPT code 71010 for ...
6,"{'id': 'CLM008', 'account': 12351, 'patient': ...",[What are the Medicare Chapter 13 rules for di...,[{'query': 'What are the Medicare Chapter 13 r...,The Medicare Chapter 13 rules for digital radi...,{'summary': 'The claim for digital radiography...


In [21]:
df_results.generated_queries[0]

['What are the Medicare Chapter 13 rules for MRI of the chest with contrast (CPT 70551) for evaluation of a lung nodule (ICD R91.8) in a hospital outpatient setting?',
 'Are there specific financial guidelines or coding requirements for approved pre-authorization claims for MRI of the chest with contrast (CPT 70551) related to lung nodules (ICD R91.8)?',
 'What are the reimbursement policies for hospital outpatient MRI services (CPT 70551) for lung nodule evaluation (ICD R91.8) under Medicare Chapter 13?']

In [22]:
df_results.claim[0]

{'id': 'CLM001',
 'account': 12345,
 'patient': 'Patient A',
 'entry_date': '2023-11-01',
 'bill_date': '2023-11-02',
 'date_of_service': '2023-10-30',
 'procedure_description': 'MRI of chest with contrast for evaluation of a lung nodule.',
 'place_of_service': 'Hospital Outpatient',
 'service_type': 'MRI',
 'units': 1,
 'provider': 'Radiology Clinic A',
 'facility': 'Hospital A',
 'cpt_codes': ['70551'],
 'icd_codes': ['R91.8'],
 'charge_amount': 1500.0,
 'allowed_amount': 1200.0,
 'status': 'submitted',
 'pre_auth_status': 'approved',
 'is_flagged': False,
 'notes': []}

In [23]:
df_results.query_metrics[0]

[{'query': 'What are the Medicare Chapter 13 rules for MRI of the chest with contrast (CPT 70551) for evaluation of a lung nodule (ICD R91.8) in a hospital outpatient setting?',
  'retrieved_answer': 'The Medicare Chapter 13 rules for MRI of the chest with contrast (CPT 70551) for evaluation of a lung nodule (ICD R91.8) in a hospital outpatient setting involve specific billing and coding guidelines. \n\nAccording to the information provided in the context, for MRI procedures in a hospital outpatient setting, the relevant HCPCS codes for Magnetic Resonance Angiography (MRA) should be reported using specific C codes instead of the standard HCPCS codes. For example, for an MRA of the chest (CPT 71555), the corresponding C codes are C8909 – C8911.\n\nAdditionally, for claims with dates of service on or after July 1, 2003, coverage under this benefit has been expanded for the use of MRA for diagnosing pathology in the renal or aortoiliac arteries. The HCPCS code to report this expanded cove